In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using Theano backend.


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

In [4]:
# Normalize the data
X_train = X_train / 255.0
test = test / 255.0
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,1,28,28)
test = test.values.reshape(-1,1,28,28)

In [5]:
Y_train = to_categorical(Y_train, num_classes = 10)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
print(X_train.shape, Y_train.shape)

(37800, 1, 28, 28) (37800, 10)


In [6]:
model = Sequential()

# conv1
model.add(Conv2D(filters = 48, kernel_size = (5,5),padding = 'same', 
                 activation ='relu', input_shape = (1,28,28)))
model.add(Conv2D(filters = 48, kernel_size = (5,5),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))
# conv2
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# flatten
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(10, activation = "softmax"))
# Compile the model

opt = Adam(lr=0.001)
model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
print("compile done")

compile done


In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

/home/jac/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3 or 4 channels on axis 1. However, it was passed an array with shape (37800, 1, 28, 28) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [ ]:
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.models import load_model

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
#2017-09-05 log
#275s - loss: 0.0475 - acc: 0.9847 - val_loss: 0.0183 - val_acc: 0.9945
#Epoch 30/60
    
best_model_path = 'best_m_v2.h5'
best_model = ModelCheckpoint(filepath=best_model_path, monitor='acc', save_best_only=True)
epochs = 40
steps_per_epoch = 100
batch_size = 90
model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=steps_per_epoch,
                              callbacks=[best_model,learning_rate_reduction])
model = load_model(best_model_path)

Epoch 1/60
273s - loss: 0.9968 - acc: 0.6604 - val_loss: 0.1493 - val_acc: 0.9529
Epoch 2/60
273s - loss: 0.2976 - acc: 0.9050 - val_loss: 0.1063 - val_acc: 0.9681
Epoch 3/60
277s - loss: 0.1939 - acc: 0.9400 - val_loss: 0.0868 - val_acc: 0.9748
Epoch 4/60
276s - loss: 0.1762 - acc: 0.9453 - val_loss: 0.0745 - val_acc: 0.9793
Epoch 5/60
266s - loss: 0.1490 - acc: 0.9549 - val_loss: 0.0542 - val_acc: 0.9833
Epoch 6/60
266s - loss: 0.1381 - acc: 0.9599 - val_loss: 0.0634 - val_acc: 0.9826
Epoch 7/60
267s - loss: 0.1289 - acc: 0.9606 - val_loss: 0.0551 - val_acc: 0.9843
Epoch 8/60
273s - loss: 0.1076 - acc: 0.9650 - val_loss: 0.0450 - val_acc: 0.9855
Epoch 9/60
279s - loss: 0.1022 - acc: 0.9694 - val_loss: 0.0457 - val_acc: 0.9864
Epoch 10/60
281s - loss: 0.0998 - acc: 0.9700 - val_loss: 0.0492 - val_acc: 0.9864
Epoch 11/60
271s - loss: 0.0974 - acc: 0.9708 - val_loss: 0.0484 - val_acc: 0.9860
Epoch 12/60
273s - loss: 0.0849 - acc: 0.9739 - val_loss: 0.0396 - val_acc: 0.9876
Epoch 13/60
2

In [ ]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
import arrow
ts = str(arrow.now())[:-10].replace(':','_').replace('.',"_")
res_file = "play_cnn_mnist_datagen_{}.csv".format(ts)

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv(res_file,index=False)

print('save done',res_file)